# Decision Tree & Random Forest Models

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/bunmiolakanmi/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna() 
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [6]:
# import os
# os.getcwd()

# Select your features (columns)

In [7]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period',  'koi_time0bk', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
selected_features.head()

,koi_period,koi_time0bk,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,54.418383,162.513840,874.8,2.83,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,19.899140,175.850252,10829.0,14.60,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,1.736952,170.307565,8079.2,33.46,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,2.525592,171.595550,603.3,2.75,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,4.134435,172.979370,686.0,2.77,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
X=selected_features
X

,koi_period,koi_time0bk,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,54.418383,162.513840,874.8,2.83,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,19.899140,175.850252,10829.0,14.60,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,1.736952,170.307565,8079.2,33.46,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,2.525592,171.595550,603.3,2.75,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,4.134435,172.979370,686.0,2.77,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,8.589871,132.016100,87.7,1.11,929,176.40,8.4,1,5638,4.296,1.088,298.74921,46.973351,14.478
6987,0.527699,131.705093,1579.2,29.35,2088,4500.53,453.3,1,5638,4.529,0.903,297.18875,47.093819,14.082
6988,1.739849,133.001270,48.5,0.72,1608,1585.81,10.6,1,6119,4.444,1.031,286.50937,47.163219,14.757
6989,0.681402,132.181750,103.6,1.07,2218,5713.41,12.3,1,6173,4.447,1.041,294.16489,47.176281,15.385


In [9]:
y = df["koi_disposition"]
target_names = ['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
X_train.head()

,koi_period,koi_time0bk,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,6.768901,133.077240,123.1,1.24,1017,253.30,10.8,1,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0.733726,132.020050,114.6,0.86,1867,2891.64,13.8,1,5855,4.578,0.797,284.50391,42.463860,15.770
2879,7.652707,134.460380,641.1,3.21,989,226.81,254.3,1,6328,4.481,0.963,295.50211,38.983540,13.099
107,7.953547,174.662240,875.4,2.25,696,55.37,38.4,1,4768,4.536,0.779,291.15878,40.750271,15.660
29,4.959319,172.258529,9802.0,12.21,1103,349.40,696.5,1,5712,4.359,1.082,292.16705,48.727589,15.263


In [12]:
y_train.head()

6122         CANDIDATE
6370    FALSE POSITIVE
2879    FALSE POSITIVE
107          CONFIRMED
29           CANDIDATE
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [13]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [14]:
# Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [16]:
# Decision Tree classifier
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_scaled, y_train)
clf.score(X_test_scaled, y_test)

0.6212814645308925

In [20]:
# Get important variables in the DT
feature_names = X.columns
sorted(zip(clf.feature_importances_, feature_names), reverse=True)

[(0.1956891285006075, 'koi_prad'),
 (0.17618838906360562, 'koi_model_snr'),
 (0.1624035558883068, 'koi_period'),
 (0.0724052569138701, 'ra'),
 (0.05333270829104247, 'koi_steff'),
 (0.053019773534108694, 'koi_time0bk'),
 (0.05219423589777835, 'dec'),
 (0.051252839630515946, 'koi_depth'),
 (0.04561185185294317, 'koi_kepmag'),
 (0.034884060685550326, 'koi_srad'),
 (0.03216656407166556, 'koi_slogg'),
 (0.028318810801059056, 'koi_teq'),
 (0.02356848581067383, 'koi_insol'),
 (0.018964339058272675, 'koi_tce_plnt_num')]

In [22]:
# Calculate classification report - DT
from sklearn.metrics import classification_report
predictions = clf.predict(X_test_scaled)
print(classification_report(y_test, predictions,
                            target_names=target_names))

                precision    recall  f1-score   support

     CONFIRMED       0.38      0.41      0.39       411
FALSE POSITIVE       0.67      0.64      0.65       484
     CANDIDATE       0.73      0.71      0.72       853

      accuracy                           0.62      1748
     macro avg       0.59      0.59      0.59      1748
  weighted avg       0.63      0.62      0.62      1748



In [17]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

0.7339816933638444

In [19]:
# Get important variables in the RF
feature_names = X.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.15206155096045243, 'koi_prad'),
 (0.1447588735701137, 'koi_model_snr'),
 (0.08876994876778857, 'koi_depth'),
 (0.0884848027299531, 'koi_period'),
 (0.07216763685117472, 'koi_insol'),
 (0.06731390196229245, 'koi_teq'),
 (0.0652279983816279, 'koi_time0bk'),
 (0.05885364026778782, 'ra'),
 (0.05343003448595194, 'koi_steff'),
 (0.049476007046469685, 'dec'),
 (0.04839785206363846, 'koi_kepmag'),
 (0.04653606374857189, 'koi_srad'),
 (0.04533726308513678, 'koi_slogg'),
 (0.01918442607904046, 'koi_tce_plnt_num')]

In [50]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test_scaled, y_test))

Test Acc: 0.614


In [21]:
# Calculate classification report - RF
from sklearn.metrics import classification_report
predictions = rf.predict(X_test_scaled)
print(classification_report(y_test, predictions,
                            target_names=target_names))

                precision    recall  f1-score   support

     CONFIRMED       0.58      0.46      0.51       411
FALSE POSITIVE       0.75      0.79      0.77       484
     CANDIDATE       0.78      0.83      0.81       853

      accuracy                           0.73      1748
     macro avg       0.70      0.69      0.70      1748
  weighted avg       0.73      0.73      0.73      1748



In [26]:
print(f"Training Data Score: {rf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.7339816933638444


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [31]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
# param_grid = {'C': [1, 5, 10],
#               'gamma': [0.0001, 0.001, 0.01]}
# gridrf = GridSearchCV(rf, param_grid, verbose=3)

In [42]:
params = {"classifier__max_depth": [3, None],
              "classifier__max_features": [1, 3, 10],
              "classifier__min_samples_split": [1, 3, 10],
              "classifier__min_samples_leaf": [1, 3, 10],
              # "bootstrap": [True, False],
              "classifier__criterion": ["gini", "entropy"]}
# gridrf = GridSearchCV(rf, param_grid, verbose=3)

In [51]:
from sklearn.pipeline import Pipeline

 #Classifier Pipeline
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer()),
#     ('classifier', RandomForestClassifier())
# ])

In [53]:
rf_grid = GridSearchCV(estimator=Pipeline , param_grid=params) #cv=10
rf_detector = rf_grid.fit(X_train_scaled, y_train)
print(rf_grid.grid_scores_)

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

In [43]:
# # Train the model with GridSearch
# grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
gridrf.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: Invalid parameter C for estimator RandomForestClassifier(n_estimators=200). Check the list of available parameters with `estimator.get_params().keys()`.

In [41]:
rf.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [ ]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
# print(grid2.best_params_)
print(grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test_scaled, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names))

# Save the Model

In [34]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'RF_MODEL.sav'
joblib.dump(rf, filename)

['RF_MODEL.sav']

In [37]:
import os
os.getcwd()

'/Users/bunmiolakanmi/Documents/UoTBootCamp/ClassActivities/UoT/Unit 21 - Sklearn/Homework/Instructions/starter_code'